In [ ]:
import pandas as pd
import os
import json

In [ ]:
colocations = pd.read_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/colocations_satbird_splot.csv")

In [ ]:
satbird = pd.read_csv("/network/projects/ecosystem-embeddings/SatBird_data_v2/USA_summer/all_summer_hotspots_final.csv")

In [ ]:
satbird = satbird.drop(columns=["Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0"])

In [ ]:
satbird["bird"] = 1

In [ ]:
coloc = colocations[["hotspot_id", "sPlot_plot"]]

In [ ]:
coloc["plant"] = 1

In [ ]:
satbird = satbird.merge(coloc, how = "outer", left_on = "hotspot_id", right_on="hotspot_id")

In [ ]:
satbird["plant"] = satbird["plant"].fillna(0).astype(int)

In [ ]:
splots = pd.read_csv("/network/projects/ecosystem-embeddings/sPlotOpen/location_data.csv")

# GET DATA IN THE US

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

In [ ]:
# Load the continental U.S. boundary shapefile
# You can get this shapefile from Natural Earth or similar resources
gdf = gpd.read_file("/network/projects/ecosystem-embeddings/US_boundaries/cb_2018_us_nation_5m.shp")


In [ ]:
multi_poly = gdf["geometry"][0]

In [ ]:
polygons = list(multi_poly.geoms)

In [ ]:
from shapely.geometry import MultiPolygon, Polygon
import matplotlib.pyplot as plt


# Plot the MultiPolygon
fig, ax = plt.subplots()
for polygon in polygons:
    x, y = polygon.exterior.xy  # Get the exterior coordinates of each polygon
    ax.plot(x, y, color='blue')
    ax.fill(x, y, color='skyblue', alpha=0.5)  # Fill the polygon with transparency

ax.set_title("MultiPolygon Plot")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
continental_us = polygons[np.argmax([p.area for p in polygons])]

In [ ]:
continental_us

In [ ]:

def is_in_us(latitude, longitude):
    point = Point(longitude, latitude)
    return continental_us.contains(point)


In [ ]:
def custom_operation(row):
    return (is_in_us(row["Latitude"], row["Longitude"]))

In [ ]:
splots["USA"] = splots.apply(custom_operation, axis = 1)

In [ ]:
splots_usa = splots[splots["USA"]]

In [ ]:
colocations

In [ ]:
plant_only = splots_usa[~splots_usa["PlotObservationID"].isin(colocations["sPlot_plot"])]

In [ ]:
plant_only["bird"] = 0

In [ ]:
plant_only = plant_only.rename(columns = {"Latitude":"lat", "Longitude":"lon", "PlotObservationID": "sPlot_plot"})

In [ ]:
len(plant_only), len(splots_usa)

In [ ]:
plant_only["split"] = "train"

In [ ]:
#ADD env data

In [ ]:
bioclim = pd.read_csv("/network/projects/ecosystem-embeddings/sPlotOpen/worldclim_data.csv")

In [ ]:
soilgrids = pd.read_csv("/network/projects/ecosystem-embeddings/sPlotOpen/soilgrid_data.csv")

In [ ]:
bioclim.columns = bioclim.columns.str.lower()
bioclim = bioclim.rename(columns = {"plotobservationid": "sPlot_plot"})
soilgrids.columns =soilgrids.columns.str.lower()
soilgrids = soilgrids.rename(columns = {"plotobservationid": "sPlot_plot"})


In [ ]:
merged_plants= plant_only.merge(soilgrids, left_on = "sPlot_plot", right_on = "sPlot_plot")

In [ ]:
merged_plants= merged_plants.merge(bioclim, left_on = "sPlot_plot", right_on = "sPlot_plot")

In [ ]:
merged_plants = merged_plants.drop(columns = ["Unnamed: 0", "unnamed: 0_x"])

In [ ]:
merged = satbird.merge(merged_plants, how = "outer")

In [ ]:
merged = merged.drop(columns = ["unnamed: 0_y", "USA"])

In [ ]:
merged.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots.csv")

# save splits

In [ ]:
train = merged[merged["split"]=="train"]
valid = merged[merged["split"]=="valid"]
test = merged[merged["split"]=="test"]

In [ ]:
coloc_test = test[test["bird"]==1]
coloc_test = coloc_test[coloc_test["plant"]==1]

In [ ]:
coloc_test.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_test_split.csv")

In [ ]:
coloc_valid = valid[valid["bird"]==1]
coloc_valid = coloc_valid[coloc_valid["plant"]==1]

In [ ]:
coloc_valid.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_valid_split_coloc.csv")

In [ ]:
valid.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_valid_split.csv")

In [ ]:
len(coloc_valid), len(coloc_test), len(valid)

In [ ]:
train.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_train_split.csv")

In [ ]:
len(train)